In [1]:
!pip install datasets --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [2]:
from datasets import load_dataset

dataset = load_dataset("Bahareh0281/liveness_images")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6427 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 6427
    })
})

In [4]:
!pip install mtcnn --q
!pip install pillow --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.5 MB/s eta 0:00:00


In [5]:
from mtcnn import MTCNN
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os

In [6]:
# def load_image_paths(dataset):
#     image_paths = []
#     labels = []
#     for example in dataset:
#         image_paths.append(example['image'])
#         labels.append(example['label'])
#     return image_paths, labels

In [9]:
def detect_and_crop_faces(image_path, detector):
    # image = Image.open(image_path)
    image_np = np.asarray(image)
    result = detector.detect_faces(image_np)
    if result:
        for person in result:
            bounding_box = person['box']
            keypoints = person['keypoints']

            # Crop the detected face
            x, y, width, height = bounding_box
            cropped_face = image_np[y:y+height, x:x+width]

            # Convert the cropped face back to an image
            cropped_face_image = Image.fromarray(cropped_face)

            return cropped_face_image
    return None

In [6]:
# Initialize the MTCNN face detector
detector = MTCNN()

In [8]:
# Define the save path
save_path = '/content/cropped'
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [ ]:
cropped_images = []
new_labels = []
i_fake = 0
i_real = 0
# Process each image and save the cropped face with the original filename
for example in dataset['train']:
    image = np.array(example['image'])
    label = example['label']
    if label == 0:
      i_fake += 1
      count = i_fake
    else:
      i_real += 1
      count = i_real
    cropped_face = detect_and_crop_faces(image, detector)
    cropped_images.append(cropped_face)
    new_labels.append(label)
    # print("Image path is:", image_path)
    if cropped_face:
        # Extract the original filename
        filename = f"{label}_{count}.jpg"
        # Save the cropped face image with the original filename
        cropped_face.save(os.path.join(save_path, filename))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 32ms/step


In [ ]:
# Create a new dataset dictionary
cropped_dataset = DatasetDict({
    'train': Dataset.from_dict({
        'image': cropped_images,
        'label': new_labels
    })
})

In [ ]:
# Print the new dataset dictionary
print(cropped_dataset)